In [4]:
from description import data_base_age,data_base_gender
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from math import floor
from sklearn import tree
import matplotlib.pyplot as plt
import pandas as pd


def generate_header_suffix(class_to_index,type_str):
    return [f'{class_} {type_str}' for class_ in class_to_index.keys()]
def save_accuracys_recalls_fscores_exactitude(confusion_matrix,class_to_index,suffix_filename):
    results = []
    numerator_exactitude = 0
    matrix_dimension_range = range(len(confusion_matrix))
    for position_class in matrix_dimension_range:
        numerator_exactitude+=(numerator := confusion_matrix[position_class][position_class])
        denominator_accuracy = 0
        denominator_recall = 0
        for position_other in range(len(confusion_matrix)):
            denominator_accuracy += confusion_matrix[position_class][position_other]
            denominator_recall += confusion_matrix[position_other][position_class]
        #print(denominator_accuracy)
        results.append( [(numerator/denominator_accuracy) if(denominator_accuracy!=0) else float('NaN'), (numerator/denominator_recall) if(denominator_recall!=0) else float('NaN')] ) 
        #print(len(results))
        results[-1].append(2*(results[-1][0]*results[-1][1])/(results[-1][0]+results[-1][1]) if (results[-1][0]+results[-1][1]!=0) else float('NaN'))
    #accuracys,recalls,fscores = results
    #print('data:',results,'index:',index)
    #return
    results.append([numerator_exactitude/sum(sum(confusion_matrix,[]),0),'',''])
    pd.DataFrame(data=results,columns=['accuracy','recall','fscore'],index=[*generate_header_suffix(class_to_index,''),'exactitude']).to_csv(f'Scores{suffix_filename}')
    
def save_confusion_matrix(confusion_matrix,columns,index,suffix_filename):
    pd.DataFrame(data = confusion_matrix,columns=columns,index=index).to_csv(f'ConfusionMatrix{suffix_filename}')
data_bases = [data_base_age,data_base_gender]
classes_to_index =[{'10s':0,'20s':1,'30s':2},{'female':0,'male':1}]
for training_rate in [.3,.6,.9]:
    print(f'training_rate:{training_rate}')
    #print(f'len{databases_classestoindex_classtypes}')
    for data_base,class_to_index,class_type in zip(data_bases,classes_to_index,['Age','Gender']): 
        print('data_base,....')   
        feature_matrix,class_list = data_base.get_feature_matrix(),data_base.get_class_list()
        class_number = len(class_to_index)
        for classifier_method,plotting_f in zip(  
            [
                MultinomialNB
                ,lambda:DecisionTreeClassifier(max_depth=10)
                ,lambda:RandomForestClassifier(max_depth=10)
                ,lambda:MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1,max_iter=500)
                ,lambda:KNeighborsClassifier(n_neighbors=3)
            ],
            [
                lambda clf: None
                ,lambda clf:tree.plot_tree(clf)
                ,lambda clf: None
                ,lambda clf: None
                ,lambda clf: None
            ]
        ):
            confusion_matrix = [[0]*class_number for _ in range(class_number)]
            clf = classifier_method()
            print('este es el clf -->',clf)
            training_length = floor(len(feature_matrix) * training_rate)

            clf.fit(feature_matrix[0:training_length],class_list[0:training_length])

            plotting_f(clf)
            plt.show()
            for predicted_class,real_class in zip(clf.predict(feature_matrix[training_length:]),class_list[training_length:]):
                #print(predicted_class,real_class)
                pI,rI = class_to_index[predicted_class],class_to_index[real_class]
                confusion_matrix[pI][rI]+=1
            columns=generate_header_suffix(class_to_index,'predicted')
            index=generate_header_suffix(class_to_index,'real')
            suffix_filename = f'_{class_type}_{str(clf).split("(")[0]}_training_rate={training_rate}_test_rate={1-training_rate}.csv'
            save_accuracys_recalls_fscores_exactitude(confusion_matrix,class_to_index,suffix_filename)
            #exactitude = a[1]
            #print(f'accuracy={accuracys},recall={recalls},exactitude={exactitude},fscore={fscores}')
            #print(confusion_matrix)
            save_confusion_matrix(confusion_matrix,columns,index,suffix_filename)



# #print(normalizedLines)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 27)